# Programming Assignment 1
#### Drew Rosales

## 1. Data Analysis

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew

First, let's look the dataset into a pandas dataframe and view the contents

In [3]:
iris_df = pd.read_csv("iris.csv")
iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


### a) Perform test statistics for each feature and class: minimum, maximum, mean, trimmed mean, alpha trimmed mean, standard deviation, skewness, and kurtosis

In [24]:
def create_test_statistics(x, p, alpha):
    n = len(x)
    x = list(sorted(x))
    minimum = np.min(x)
    maximum = np.max(x)
    mu = np.mean(x, axis=0) # column-wise mean
    mu_trim = np.mean(x[p:-p], axis=0)
    mu_alpha = np.mean(x[int(np.floor(n*alpha)) :-int(np.floor(n*alpha))], axis=0) 
    sigma = np.std(x, axis=0)
    skewness = skew(x, axis=0)
    kurt = kurtosis(x, axis=0)
    return (minimum, maximum, mu, mu_trim, mu_alpha, sigma, skewness, kurt)
    

Separate per class into `setosa`, `virginica` and `versicolor` for each of the features

In [25]:
iris_table = [[iris_df[feature].loc[iris_df["species"] == class_name] for class_name in ["setosa", "versicolor", "virginica"]] for feature in ["sepal_length", "sepal_width", "petal_length", "petal_width"]]
iris_table[3][2]

100    2.5
101    1.9
102    2.1
103    1.8
104    2.2
105    2.1
106    1.7
107    1.8
108    1.8
109    2.5
110    2.0
111    1.9
112    2.1
113    2.0
114    2.4
115    2.3
116    1.8
117    2.2
118    2.3
119    1.5
120    2.3
121    2.0
122    2.0
123    1.8
124    2.1
125    1.8
126    1.8
127    1.8
128    2.1
129    1.6
130    1.9
131    2.0
132    2.2
133    1.5
134    1.4
135    2.3
136    2.4
137    1.8
138    1.8
139    2.1
140    2.4
141    2.3
142    1.9
143    2.3
144    2.5
145    2.3
146    1.9
147    2.0
148    2.3
149    1.8
Name: petal_width, dtype: float64

`iris_table` is indexed first by `feature` such as `sepal_length`, it is then indexed by `class` like `setosa`. Above we see, `iris_table[3][2]` this notates `petal_width` for the `virginica` class

Each test statistic is represented as a tuple in the following format: `(minimum, maximum, mean, trimmed mean, alpha trimmed mean, standard deviation, skewness, kurtosis)`. A tuple is placed in a list in the following order for the features of the three species: `[sepal_length, sepal_width, petal_length, petal_width]`.

In [27]:
iris_stats = [[create_test_statistics(class_vector.values, 5, 0.2) for class_vector in feature_vector] for feature_vector in iris_table]
iris_stats[0]

[(np.float64(4.3),
  np.float64(5.8),
  np.float64(5.006),
  np.float64(5.0024999999999995),
  np.float64(4.999999999999999),
  np.float64(0.3489469873777391),
  np.float64(0.11645392749203061),
  np.float64(-0.34576467558246327)),
 (np.float64(4.9),
  np.float64(7.0),
  np.float64(5.935999999999999),
  np.float64(5.9375),
  np.float64(5.910000000000001),
  np.float64(0.5109833656783751),
  np.float64(0.10218956951841443),
  np.float64(-0.5988273407672811)),
 (np.float64(4.9),
  np.float64(7.9),
  np.float64(6.587999999999999),
  np.float64(6.572500000000001),
  np.float64(6.546666666666668),
  np.float64(0.6294886813914926),
  np.float64(0.11444474247296789),
  np.float64(-0.08794223159166314))]

Here we trimmed the first and last values since `p=5` and also first and last 5% since `alpha=0.02`. Above we see `iris_stats[0]` which represents the test statistics for `sepal_length` under all three classes

### b) Create Analysis and an explanation for each test statistic. What conclusions can you make based on this data?

The first two values in statistics are `minimum` and `maximum`, which indicate the smallest and largest values in the vector respectively. For `sepal_length` above we can see that `setosa` has the smallest range with `[4.3, 5.8]` while `versicolor` and `virginica` have the similar range of `[4.9, 7.0]` and `[4.9, 7.9]` respectively. Next we can look at the next three statistics: `mean`, `trimmed mean`, and `alpha trimmed mean`. These show the average values within the features but `trimmed mean` will disregard the smallest and largest `p` values in the feature. Similarly, `alpha trimmed mean` will take a percentage off the extremes of the feature vector. Given our features are `50 x 1` then `n=50`. With a value of `alpha=0.05`, 20% of 50 yields 10 observations trimmed off the front and back. The differences in these variants of means can be an indicator of how centered our observations are relative to the mean. If there is a particular large value and/or outlier then the mean may be skewed to the left. Similarly, if there is a small value/outlier then the observation may be skewed right. My trimming the mean we expect to get closer to the median of the feature, which is more resilient to outliers. Within the `sepal_length` feature for the `setosa` class we can see the mean holds at `5.006` while the trimmed mean (5 values off either side) yields `5.002`. This gives us an indicator that the trimmed values do not skew the overal feature vector since there is only a difference in the order of `10^-3`. We can see that the `alpha trimmed mean` for this feature and class also holds a similar results but to a lesser magnitude, `10^-1`.